In [8]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_csv('users_behavior.csv')
df.sample(20)


,calls,minutes,messages,mb_used,is_ultra
3166,71.0,506.27,98.0,19864.42,0
1471,53.0,405.98,0.0,24831.39,0
1846,25.0,141.99,84.0,16150.12,1
2924,106.0,680.88,2.0,23760.26,0
477,47.0,322.44,11.0,11420.07,0
1981,15.0,152.00,14.0,5144.73,0
2634,74.0,472.44,18.0,16081.14,0
43,50.0,323.99,10.0,20922.05,1
417,74.0,539.51,27.0,7732.23,1
2477,68.0,487.59,45.0,13544.52,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Empezaremos con definir las caracteristicas y el objetivo, empezando con una regresion lineal para ver su remc.

In [4]:
features = df[['calls', 'minutes', 'messages', 'mb_used']] 
target = df['is_ultra'] 

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25,random_state=12345)


Procedo a usar el modelo LogisticRegresion, para empezar a comparar modelos.

In [22]:
model = LogisticRegression(random_state=54321, solver='liblinear') # inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(features_train, target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.74149377593361
Accuracy del modelo de regresión logística en el conjunto de validación: 0.753731343283582


Dados los resultados podemos observar que en comparacion al decisiontree y al randomforest es muy baja la exactitud.

Haremos un decisiontreeclassifier porque es una tarea de clasificacion.

In [24]:
for depth in range(1,6):  # selecciona el rango del hiperparámetro
    # < escribe tu código aquí >
    model=DecisionTreeClassifier(max_depth=depth,random_state=54321)
    model.fit(features_train,target_train)
    test_p=model.predict(features_valid)
    acc=accuracy_score(target_valid,test_p)
    print("Exactitud de max_depth igual a", depth)
    print("Conjunto de entrenamiento:", acc) # calcula la puntuación de accuracy en el conjunto de entrenamiento
    print("Conjunto de prueba:", accuracy_score(target_valid, test_p)) # calcula la puntuación de accuracy en el conjunto de prueba
    print()

Exactitud de max_depth igual a 1
Conjunto de entrenamiento: 0.75
Conjunto de prueba: 0.75

Exactitud de max_depth igual a 2
Conjunto de entrenamiento: 0.7835820895522388
Conjunto de prueba: 0.7835820895522388

Exactitud de max_depth igual a 3
Conjunto de entrenamiento: 0.7885572139303483
Conjunto de prueba: 0.7885572139303483

Exactitud de max_depth igual a 4
Conjunto de entrenamiento: 0.7810945273631841
Conjunto de prueba: 0.7810945273631841

Exactitud de max_depth igual a 5
Conjunto de entrenamiento: 0.7810945273631841
Conjunto de prueba: 0.7810945273631841



Ya que tenemos los resultados de estos, tenemos observaciones de como con la profundidad aumentando la exactitud del modelo empieza a fallar, siendo la profundidad 3 la que mejor a funcionado. Hare un modelo final para dejar la prueba.

In [19]:
final_model = DecisionTreeClassifier(max_depth=3, random_state=54321)

# Combinar datos de entrenamiento y validación para entrenar el modelo final
features_combined = pd.concat([features_train, features_valid])
target_combined = pd.concat([target_train, target_valid])

# Entrenar el modelo
final_model.fit(features_combined, target_combined)

# Evaluación del modelo final (si tienes un conjunto de prueba independiente):
# test_predictions = final_model.predict(features_test)
# test_accuracy = accuracy_score(target_test, test_predictions)
# print("Exactitud en el conjunto de prueba:", test_accuracy)

print("Modelo final entrenado con max_depth=3")

Modelo final entrenado con max_depth=3


Siguiente de lo anterior procedemos a crear un bosque aleatorio

In [25]:
best_score = 0
best_est = 0
for est in range(1, 11): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est) # configura el número de árboles
    model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    score = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score:
        best_score = score# guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 4): 0.7885572139303483


Comparando 